# Visualizing ZOD in FiftyOne

The ZOD is a large multi-modal autonomous driving dataset collected over a 2-year period in 14 different European countries, designed to support various aspects of autonomous driving research. This script facilitates the conversion of ZOD into FiftyOne format, enabling users to leverage FiftyOne's powerful tools for dataset exploration and analysis.

In this notebook we will instructions on how to use the [Zenseact Open Dataset (ZOD)](https://zod.zenseact.com) with [FiftyOne](https://voxel51.com), a powerful open-source tool for computer vision dataset exploration and analysis. 

![](./docs/zod-fiftyone.gif)

## Requirements

Please install the packages in `requirements.txt` before getting started:
```
zod==0.3.6 
fiftyone==0.23.2
open3d==0.17.0
pyyaml==6.0.1 
tqdm==4.66.1
```

Once the required packages are installed, we will check for any existing datasets.

In [ ]:
import fiftyone as fo

fo.list_datasets()

If this list is empty, we will start by importing ZOD into fiftyone by creating a dataset.

## Create a new dataset

With the requirements already installed, we have to:

### 1. Update [`config.yaml`](./src/config.yaml)

Specify the necessary configurations for your dataset. Here's a breakdown of the configuration options:

- `dataset_root`: Point this to the root directory of the ZOD dataset, if you haven't downloaded it yet, see the download options [here](https://zod.zenseact.com/download/).
- `pcd_files_dir`: Directory where converted PCD files will be stored. If it does not already exist, it will be created.
- `dataset_version`: Specify whether you're using the "mini" or "full" version of the dataset.
- `dataset_split`: Choose the dataset split ("train", "val", or "all"). For the "mini" version, please use "all".
- `dataset_name`: The name you want to give to your dataset.
- `dataset_persistent`: Set to True if you want to save the dataset locally for later use (recommended).
- `test_run`: Set to True for a test run on the first 10 samples.
- `mapbox_token`: Optional. Provide a [Mapbox](https://www.mapbox.com) API token for map functionality in FiftyOne (you can create one for free [here](https://account.mapbox.com/access-tokens/create)).

### 2. Run [`zod_to_fiftyone.py`](./src/zod_to_fiftyone.py)

Please note that due to a FiftyOne limitation, we need to create `.pcd` files to visualize point clouds in the app. This conversion process will take a while due the size of the dataset.

## Load the dataset

Once you have created the datasets, choose an existing one from the list to load it. 

In [ ]:
import fiftyone as fo

fo.list_datasets()

In [ ]:
dataset = fo.load_dataset('zod_val')

In [ ]:
dataset

## Launch the app

You can use the app inside a notebook, but also launch in your browser. Learn more [here](https://docs.voxel51.com/user_guide/app.html).

In [ ]:
session = fo.launch_app(dataset)

## Add new field to each sample

This will take a few minutes to run depending on the size of the dataset.

In [ ]:
for sample in dataset.iter_samples(progress=True):
    sample["split"] = "Validation"
    sample.save()

In [ ]:
dataset.media_type

You can read more about grouped datasets in the FiftyOne documentation [here](https://docs.voxel51.com/user_guide/groups.html).

In [ ]:
dataset.stats(include_media=True)

## Get random sample

This is useful to see the structure of your samples.

In [ ]:
sample = dataset.take(1).first()
sample

In [ ]:
# lists the names of the fields in the sample
sample.field_names

## Get coordinates for random samples

In [ ]:
dataset.take(5).values("location.point", _raw=True)

## Filtering

You can easily filter the samples in your dataset in the app GUI, but you can also do it programmatically. Here are some examples.

### by label

In [ ]:
from fiftyone import ViewField as F

In [ ]:
just_animals = dataset.filter_labels(
    "detections",
    F("label").contains_str("Animal"),
)

You can also count how many samples exist in the filtered view we just created. You can read more about views in FiftyOne [here](https://docs.voxel51.com/user_guide/using_views.html).

In [ ]:
# count of animal labels in the filtered view
just_animals.count("detections")

### by file path

In [ ]:
dataset.match(F("filepath").contains_str("india"))

## Saving a view

In [ ]:
# will save a view with all samples that contain an animal
dataset.save_view("detections_animals", just_animals)

## Export view as KITTI dataset

You can export data in many common formats (YOLO, KITTI, COCO, etc.) as well as having the option to build [custom exporters](https://docs.voxel51.com/user_guide/export_datasets.html#custom-formats). Here is an example of exporting a view as a KITTI object detection dataset.

In [ ]:
import os

export_dir = "./kitti"
label_field = "detections"

os.makedirs(export_dir, exist_ok=True)

just_animals.export(
    export_dir=export_dir,
    dataset_type=fo.types.KITTIDetectionDataset,
    label_field=label_field,
)

This is just a preview on how FiftyOne can be used to interact with the ZOD dataset programmatically. For more information please refer to their [documentation](https://docs.voxel51.com).